In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Data Exploration
I want to scrape the Purdue butcher block website to see when they update it, scrape the results, and provide an easily navigatable interface (and tell me if they have chuck eyes)

In [3]:
url = 'https://ag.purdue.edu/ansc/ButcherBlock/Pages/Meatlist.aspx'
html = requests.get(url).text

All of the values I want to parse are located in \<tr\> elements, so I can use BeautifulSoup to parse out the table rows

In [4]:

soup = BeautifulSoup(html,'html.parser')
rows = soup.find_all('tr')

Thhe values are nested in \<td\> elements inside of the table rows, in an attribute called data-sheets-value. I can extract each of these into a list

In [5]:
import json
for row in rows:
    curr = []
    for td in row:
        if 'data-sheets-value' in td.attrs:
            #print(td['data-sheets-value'])
            val = json.loads(td['data-sheets-value'])
            if '2' in val:
                curr.append(val['2'])
            else:
                curr.append(val['3'])
    print(curr)

['   Boilermaker Butcher Block Meat List February 16th, 17th & 18th, 2022']
['Wednesday, Thursday & Friday,  11AM-5:30PM', '* Indicates Sales Items']
[]
['Beef Steaks', 'Average Weight', 'Price Per Pound', 'Pork Chops, Steaks & Ribs', 'Average Weight', 'Price Per Pound']
['Top Round', 2, 4.99, 'Fresh Ham Steak', 1, 1.89]
['Eye of Round', 1, 5.99, "Boneless America's Chops", 1.5, 3.99]
['Sirloin TIp Steak', 1, 7.99, 'Blade Steaks', 1.5, 1.49]
['Top Sirloin', 1, 8.99, ' ']
['Flat Iron', 1, 8.99, 'Pork Roasts', 'Average Weight', 'Price Per Pound']
['Bone-In Ribeye', 1.5, 15.99, 'Fresh Ham Roast', 3, 0.99]
['NY Stripp', 1, 17.99, 'Butt Roast', 10, 1.99]
['Boneless Ribeyes', 1, 20.99, 'Boneless Pork Loin', 43865, 2.49]
['Picnic Roast', 5, 0.99]
['Beef Roasts', 'Average Weight', 'Price Per Pound']
['*Short Ribs', 3, 2.99, 'Ground Products', 'Average Weight', 'Price Per Pound']
['Chuck Arm Roast', 3, 3.99, 'Bacon Cheeseburgers', 5, 7.99]
['Boneless Chuck Roast', 2, 4.49, 'Ground Round ', 1, 6

It gets a little complicated, but basically the header is Category | Avg Weight | $/lb
The rows arent always lined up header to header, so I need to dynamically figure out when the headers per row are starting/stopping

In [34]:
import json
h1 = None
h2 = None
d1 = {}
run1 = []
run2 = []
for i,row in enumerate(rows[3:]):
    tds = [td.string for td in row if 'data-sheets-value' in td.attrs]
    print(tds)
    #print(run1)
    vals = [td for td in tds]
    vals = [v for v in vals if v != ' '] 
    #print(f'current row vals:{vals}')
    col1,col2 = vals[:3],vals[3:]
    #print(f'col1:{col1}\ncol2:{col2}')
    #print(f'run1:{run1}\nrun2:{run2}')
    if 'Average Weight' in col1:
        if h1 == None:
            h1 = col1[0]
        else:
            d1[h1] = run1
            h1 = col1[0]
            run1 = []
    elif col1!=[] and col2 != []:
        run1.append(col1)
    elif col2 == []:
        run2.append(col1)
    if 'Average Weight' in col2:
        if h2 == None:
            h2 = col2[0]
        else:
            d1[h2] = run2
            h2 = col2[0]
            run2 = []
    elif col2 != None:
        run2.append(col2)




['Beef Steaks', 'Average Weight', 'Price Per Pound', 'Pork Chops, Steaks & Ribs', 'Average Weight', 'Price Per Pound']
['Top Round', '2', '$4.99', 'Fresh Ham Steak', '1', '$1.89']
['Eye of Round', '1', '$5.99', "Boneless America's Chops", '1.5', '$3.99']
['Sirloin TIp Steak', '1', '$7.99', 'Blade Steaks', '1.5', '$1.49']
['Top Sirloin', '1', '$8.99', None]
['Flat Iron', '1', '$8.99', 'Pork Roasts', 'Average Weight', 'Price Per Pound']
['Bone-In Ribeye', '1.5', '$15.99', 'Fresh Ham Roast', '3', '$0.99']
['NY Stripp', '1', '$17.99', 'Butt Roast', '10', '$1.99']
['Boneless Ribeyes', '1', '$20.99', 'Boneless Pork Loin', '2-4', '$2.49']
['Picnic Roast', '5', '$0.99']
['Beef Roasts', 'Average Weight', 'Price Per Pound']
['*Short Ribs', '3', '$2.99', 'Ground Products', 'Average Weight', 'Price Per Pound']
['Chuck Arm Roast', '3', '$3.99', 'Bacon Cheeseburgers', '5', '$7.99']
['Boneless Chuck Roast', '2', '$4.49', 'Ground Round ', '1', '$6.99']
['Sirloin Tip Roast', '2', '$4.99', 'Lamb Patties

I got it, besides some additional empty lists that were added to the dictionary. I can now create a pandas df 

In [64]:
columns = ['item','avg weight/size','price/lb|unit']
df_dict = {k : pd.DataFrame(v,columns=columns) for k,v in d1.items()}

df = (pd.concat(df_dict)
        .reset_index(level=1,drop=True)
        .rename_axis('category')
        .reset_index()
        .dropna()
)

In [65]:
df

,category,item,avg weight/size,price/lb|unit
0,"Pork Chops, Steaks & Ribs",Fresh Ham Steak,1,$1.89
1,"Pork Chops, Steaks & Ribs",Boneless America's Chops,1.5,$3.99
2,"Pork Chops, Steaks & Ribs",Blade Steaks,1.5,$1.49
4,Beef Steaks,Top Round,2,$4.99
5,Beef Steaks,Eye of Round,1,$5.99
6,Beef Steaks,Sirloin TIp Steak,1,$7.99
7,Beef Steaks,Top Sirloin,1,$8.99
8,Beef Steaks,Flat Iron,1,$8.99
9,Beef Steaks,Bone-In Ribeye,1.5,$15.99
10,Beef Steaks,NY Stripp,1,$17.99


Got it into a dataframe, now I want to be able to select category, and sort by weight/price.

I also will identify sale items, and put those on the main page

In [67]:
df = df.applymap(str)
def weight_range_to_avg(s):
    if '-' in s:
        vals = s.split('-')
        return sum([int(val) for val in vals])/len(vals)
    return float(s)
def dollar_string_to_float(s):
    if '$' in s:
        return float(s[1:])
    return float(s)

df['avg weight/size'] = df['avg weight/size'].map(weight_range_to_avg)
df['price/lb|unit'] = df['price/lb|unit'].map(dollar_string_to_float)



In [ ]:
df

In [69]:
df.to_csv('butcher_block02-17.csv')